In [1]:
import csv
import pandas as pd
import re

#### Life expectancy

In [2]:
life_exp_2019 = "data\health\life_2012_2019_data.csv"
life_exp_2020 = "data\health\life_2020_data.csv"

In [3]:
life_exp_2019 = pd.read_csv(life_exp_2019)
life_exp_2019

,measure_id,measure_name,location_id,location_name,state_name,race_ethnicity_group,sex_id,sex_name,age_group_id,age_group_name,year_id,metric_id,metric_name,val,upper,lower
0,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,73,25,1990,5,Years,47.401899,47.663865,47.131186
1,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,73,25,1991,5,Years,47.293256,47.535055,47.033662
2,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,73,25,1992,5,Years,47.667564,47.935554,47.409149
3,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,73,25,1993,5,Years,47.826836,48.098470,47.571233
4,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,73,25,1994,5,Years,47.718326,47.982345,47.456253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627295,14,All-cause mortality rate,53912,"Wyoming; Non-Hispanic, White",Wyoming,"Non-Hispanic, White",2,Female,160,85 plus,2015,3,Rate,0.130016,0.134438,0.125880
627296,14,All-cause mortality rate,53912,"Wyoming; Non-Hispanic, White",Wyoming,"Non-Hispanic, White",2,Female,160,85 plus,2016,3,Rate,0.122373,0.126349,0.118465
627297,14,All-cause mortality rate,53912,"Wyoming; Non-Hispanic, White",Wyoming,"Non-Hispanic, White",2,Female,160,85 plus,2017,3,Rate,0.124795,0.129249,0.120578
627298,14,All-cause mortality rate,53912,"Wyoming; Non-Hispanic, White",Wyoming,"Non-Hispanic, White",2,Female,160,85 plus,2018,3,Rate,0.132197,0.136809,0.127338


In [4]:
life_2019 = life_exp_2019.loc[life_exp_2019['measure_name'] == 'Life expectancy']\
    [life_exp_2019['age_group_name'] == 'Birth'][life_exp_2019['year_id'] >= 2011]\
    [life_exp_2019['race_ethnicity_group'] >= 'State-level']
life_2019

C:\Users\18973\AppData\Local\Temp\ipykernel_25176\65814213.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  life_2019 = life_exp_2019.loc[life_exp_2019['measure_name'] == 'Life expectancy']\


,measure_id,measure_name,location_id,location_name,state_name,race_ethnicity_group,sex_id,sex_name,age_group_id,age_group_name,year_id,metric_id,metric_name,val,upper,lower
141,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,164,Birth,2011,5,Years,72.763545,73.028044,72.477746
142,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,164,Birth,2012,5,Years,72.792684,73.066355,72.506043
143,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,164,Birth,2013,5,Years,72.627114,72.893336,72.372241
144,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,164,Birth,2014,5,Years,73.008197,73.280054,72.753815
145,26,Life expectancy,523,Alabama,Alabama,State-level,1,Male,164,Birth,2015,5,Years,72.705515,72.998800,72.438260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9175,26,Life expectancy,573,Wyoming,Wyoming,State-level,2,Female,164,Birth,2015,5,Years,81.172748,81.662615,80.658190
9176,26,Life expectancy,573,Wyoming,Wyoming,State-level,2,Female,164,Birth,2016,5,Years,81.631941,82.165660,81.089740
9177,26,Life expectancy,573,Wyoming,Wyoming,State-level,2,Female,164,Birth,2017,5,Years,81.358672,81.879027,80.809047
9178,26,Life expectancy,573,Wyoming,Wyoming,State-level,2,Female,164,Birth,2018,5,Years,81.546103,82.103023,80.992780


In [5]:
def gender_to_ratio(df):
    new_df = df[df['sex_name'] == "Female"][["state_name", "sex_name", "year_id", "val"]].reset_index()
    m_df = df[df['sex_name'] == "Male"][["state_name", "sex_name", "year_id", "val"]].reset_index()
    new_df['mtf_ratio'] = m_df['val'].divide(new_df['val'])
    new_df = new_df.drop(columns=['sex_name']).reset_index()
    return new_df

In [6]:
life_df_2019 = gender_to_ratio(life_2019)
life_df_2019 = life_df_2019.drop(columns='index')
life_df_2019

,level_0,state_name,year_id,val,mtf_ratio
0,0,Alabama,2011,78.331475,0.928918
1,1,Alabama,2012,78.166435,0.931252
2,2,Alabama,2013,78.321583,0.927294
3,3,Alabama,2014,78.247415,0.933043
4,4,Alabama,2015,78.175457,0.930030
...,...,...,...,...,...
454,454,Wyoming,2015,81.172748,0.947130
455,455,Wyoming,2016,81.631941,0.945792
456,456,Wyoming,2017,81.358672,0.955632
457,457,Wyoming,2018,81.546103,0.946401


Combine 2020 data

In [7]:
''' 
data from a pdf: 
        convert to Excel first, do some data wrangling 
        export as a csv
'''
life_2020 = pd.read_csv(life_exp_2020)
life_2020 = life_2020.drop(columns=life_2020.columns[-1]).reset_index()
life_2020['year_id'] = life_2020['year_id'].astype(int)
life_2020.head()

,index,state_name,val,sex_name,year_id
0,0,Alabama,70.1,Male,2020
1,1,Alabama,76.4,Female,2020
2,2,Alaska,74.3,Male,2020
3,3,Alaska,79.2,Female,2020
4,4,Arizona,73.4,Male,2020


In [8]:
life_df_2020 = gender_to_ratio(life_2020)
life_df_2020.drop(columns='index', inplace=True)
life_df_2020.drop(44, inplace=True)   # remove state == "United States"
life_df_2020.reset_index(drop=True)

,level_0,state_name,year_id,val,mtf_ratio
0,0,Alabama,2020,76.4,0.917539
1,1,Alaska,2020,79.2,0.938131
2,2,Arizona,2020,79.5,0.923270
3,3,Arkansas,2020,76.6,0.928198
4,4,California,2020,82.0,0.929268
5,5,Colorado,2020,80.9,0.936959
6,6,Connecticut,2020,81.3,0.929889
7,7,Delaware,2020,79.5,0.929560
8,8,District of Columbia,2020,78.7,0.911055
9,9,Florida,2020,80.5,0.926708


In [9]:
life_exp = pd.concat([life_df_2019, life_df_2020]).reset_index()
life_exp = life_exp.drop(columns=['index','level_0'])
life_exp

,state_name,year_id,val,mtf_ratio
0,Alabama,2011,78.331475,0.928918
1,Alabama,2012,78.166435,0.931252
2,Alabama,2013,78.321583,0.927294
3,Alabama,2014,78.247415,0.933043
4,Alabama,2015,78.175457,0.930030
...,...,...,...,...
505,Virginia,2020,80.100000,0.937578
506,Washington,2020,81.600000,0.942402
507,West Virginia,2020,76.100000,0.917214
508,Wisconsin,2020,80.300000,0.936488


In [10]:
life_exp = life_exp.sort_values(['state_name', 'year_id'], ascending=[True, True])
life_exp.head(20)

,state_name,year_id,val,mtf_ratio
0,Alabama,2011,78.331475,0.928918
1,Alabama,2012,78.166435,0.931252
2,Alabama,2013,78.321583,0.927294
3,Alabama,2014,78.247415,0.933043
4,Alabama,2015,78.175457,0.930030
5,Alabama,2016,78.139206,0.928596
6,Alabama,2017,78.373311,0.927302
7,Alabama,2018,78.347347,0.927866
8,Alabama,2019,78.671102,0.926461
459,Alabama,2020,76.400000,0.917539


In [103]:
life_exp.to_csv("life_exp.csv", encoding='utf-8', index=False)

#### Sex ratio (males per 100 females)

In [11]:
import re

In [12]:
def remove_after_exclamation(s):
    return re.sub('!.+', '', s)

In [89]:
def get_sr_df(df, year):
    ''' Transpose a row to a dataframe '''
    sr_df = df[df['Label (Grouping)'].str.contains(r'Sex ratio')].T.reset_index() # get the specific row
    sr_df.columns = sr_df.iloc[0]
    sr_df = sr_df.drop(sr_df.index[0])   # reset column names
    sr_df.columns = {'state_name': 'Label (Grouping)', 'sex_ratio': 'Sex ratio (males per 100 females)'}
    sr_df['state_name'] = sr_df['state_name'].apply(remove_after_exclamation)
    sr_df.drop(sr_df[(sr_df['sex_ratio'] == '(X)')].index, inplace=True)
    sr_df.drop(sr_df[(sr_df['state_name'] == 'District of Columbia')].index, inplace=True)
    sr_df.drop(sr_df[(sr_df['state_name'] == 'Puerto Rico')].index, inplace=True)
    sr_df['year'] = year
       
    return sr_df.reset_index(drop=True)

In [90]:
sr_2011 = pd.read_csv("data\health\sex_ratio\sr_2011.csv")
sr_2011_df = get_sr_df(sr_2011, 2011)
sr_2011_df

,state_name,sex_ratio,year
0,Alabama,93.8,2011
1,Alaska,106.6,2011
2,Arizona,99.2,2011
3,Arkansas,97.2,2011
4,California,99.0,2011
5,Colorado,100.8,2011
6,Connecticut,94.9,2011
7,Delaware,94.2,2011
8,Florida,95.7,2011
9,Georgia,96.1,2011


In [99]:
sr_2012 = pd.read_csv('data\health\sex_ratio\sr_2012.csv')
sr_2012_df = get_sr_df(sr_2012, 2012)
sr_2013 = pd.read_csv('data\health\sex_ratio\sr_2013.csv')
sr_2013_df = get_sr_df(sr_2013, 2013)
sr_2014 = pd.read_csv('data\health\sex_ratio\sr_2014.csv')
sr_2014_df = get_sr_df(sr_2014, 2014)
sr_2015 = pd.read_csv('data\health\sex_ratio\sr_2015.csv')
sr_2015_df = get_sr_df(sr_2015, 2015)
sr_2016 = pd.read_csv('data\health\sex_ratio\sr_2016.csv')
sr_2016_df = get_sr_df(sr_2016, 2016)
sr_2017 = pd.read_csv('data\health\sex_ratio\sr_2017.csv')
sr_2017_df = get_sr_df(sr_2017, 2017)
sr_2018 = pd.read_csv('data\health\sex_ratio\sr_2018.csv')
sr_2018_df = get_sr_df(sr_2018, 2018)
sr_2019 = pd.read_csv('data\health\sex_ratio\sr_2019.csv')
sr_2019_df = get_sr_df(sr_2019, 2019)
sr_2020 = pd.read_csv('data\health\sex_ratio\sr_2020.csv')
sr_2020_df = get_sr_df(sr_2020, 2020)

In [100]:
# Concat all dfs
sex_ratio = pd.concat([sr_2011_df, sr_2012_df, sr_2013_df, sr_2014_df, sr_2015_df,
    sr_2016_df, sr_2017_df, sr_2018_df, sr_2019_df, sr_2020_df]).reset_index(drop=True)
sex_ratio

,state_name,sex_ratio,year
0,Alabama,93.8,2011
1,Alaska,106.6,2011
2,Arizona,99.2,2011
3,Arkansas,97.2,2011
4,California,99.0,2011
...,...,...,...
495,Virginia,96.8,2020
496,Washington,100.1,2020
497,West Virginia,97.8,2020
498,Wisconsin,99.1,2020


In [104]:
sex_ratio.to_csv("sex_ratio.csv", encoding='utf-8', index=False)